In [46]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline, PipelineModel

In [47]:
spark = SparkSession.builder \
.appName("PipelineOps") \
.master("local[2]") \
.config("spark.executor.memory","4g") \
.config("spark.driver.memory","2g") \
.getOrCreate()

In [48]:
df = spark.read \
.option("header","True") \
.option("sep",",") \
.option("inferSchema","True") \
.format("csv") \
.load("D:/Datasets/simple_data.csv")

In [49]:
df.toPandas().head()

,sirano,isim,yas,meslek,sehir,aylik_gelir
0,1,Cemal,35,Isci,Ankara,3500
1,2,Ceyda,42,Memur,Kayseri,4200
2,3,Timur,30,Müzisyen,Istanbul,9000
3,4,Burcu,29,Pazarlamaci,Ankara,4200
4,5,Yasemin,23,Pazarlamaci,Bursa,4800


# VERİ SETİNE ETİKET EKLEME

In [50]:
df1 = df.withColumn("ekonomik_durum",
    F.when(F.col("aylik_gelir") > 7000, "iyi").otherwise("kötü")
)

# VERİ SETİNİ TRAIN-TEST AYIRMA

In [51]:
train_df, test_df = df1.randomSplit([0.8, 0.2], seed=142)

# Pipeline için nesneleri oluşturma

In [52]:
meslek_indexer = StringIndexer() \
.setInputCol("meslek") \
.setOutputCol("meslek_index") \
.setHandleInvalid("skip")

In [53]:
sehir_indexer = StringIndexer() \
.setInputCol("sehir") \
.setOutputCol("sehir_index") \
.setHandleInvalid("skip")

In [54]:
encoder = OneHotEncoderEstimator() \
.setInputCols(["meslek_index","sehir_index"]) \
.setOutputCols(["meslek_encoded","sehir_encoded"])

In [55]:
assembler = VectorAssembler() \
.setInputCols(["yas","aylik_gelir","meslek_encoded","sehir_encoded"]) \
.setOutputCol("vectorized_features")

In [56]:
label_indexer = StringIndexer() \
.setInputCol("ekonomik_durum") \
.setOutputCol("label")

In [57]:
scaler = StandardScaler() \
.setInputCol("vectorized_features") \
.setOutputCol("features")

In [59]:
lr_object = LogisticRegression() \
.setFeaturesCol("features") \
.setLabelCol("label") \
.setPredictionCol("prediction")

# Pipeline

In [60]:
pipeline_nesnesi = Pipeline() \
.setStages([meslek_indexer,sehir_indexer,encoder,assembler,label_indexer,scaler,lr_object])

In [61]:
pipeline_modeli = pipeline_nesnesi.fit(train_df)

In [62]:
pipeline_modeli.transform(test_df).select("label","prediction").toPandas().head()

,label,prediction
0,1.0,1.0
1,0.0,0.0
2,1.0,1.0
